In [2]:

# 기본 라이브러리
import numpy as np
import pandas as pd
import seaborn as sns

# 그래프 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# 기본 설정 

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
mpl.style.use("seaborn-v0_8")
mpl.rcParams.update({"figure.constrained_layout.use": True})
sns.set_context("paper") 
sns.set_palette("Set2") 
sns.set_style("whitegrid") 

plt.rc("font", family = "Malgun Gothic")
plt.rcParams["axes.unicode_minus"] = False

In [3]:
import os
os.getcwd()

'c:\\Users\\Administrator\\Desktop\\experiment\\code'

In [ ]:
# 경로 설정 = json 4개의 파일 경로 입력 // 따로 열진 않음. 
import json

with open("../data/image_data/train/label/train/train_anger.json", encoding="cp949") as f:
    data1 = json.load(f)
with open("../data/image_data/train/label/train/train_happy.json", encoding="cp949") as f:
    data2 = json.load(f)
with open("../data/image_data/train/label/train/train_panic.json", encoding="cp949") as f:
    data3 = json.load(f)
with open("../data/image_data/train/label/train/train_sadness.json", encoding="cp949") as f:
    data4 = json.load(f)

In [ ]:

# 파일 수와 폴더 목록을 가져오는 함수
import os

def count_files_in_directory(directory_path):
    try:
        # 디렉토리 안의 파일과 폴더 목록을 가져옵니다.
        entries = os.listdir(directory_path)
        # 파일만 카운트합니다.
        file_count = sum(1 for entry in entries if os.path.isfile(os.path.join(directory_path, entry)))
        return file_count
    except FileNotFoundError:
        print("지정한 경로가 존재하지 않습니다.")
        return None

In [ ]:
# 경로는 자유롭게 변경하여 사용. 하되, json 파일 위의 파일 경로를 정확히 한다. 
train_directory_path = '../data/image_data/train/img/train/'
print(f"화남 파일 개수: {count_files_in_directory(train_directory_path + 'anger'):4d} ,json 딕셔너리 갯수 {len(data1):4d}")
print(f"행복 파일 개수: {count_files_in_directory(train_directory_path + 'happy'):4d} ,json 딕셔너리 갯수 {len(data2):4d}")
print(f"패닉 파일 개수: {count_files_in_directory(train_directory_path + 'panic'):4d} ,json 딕셔너리 갯수 {len(data3):4d}")
print(f"슬픔 파일 개수: {count_files_in_directory(train_directory_path + 'sadness'):4d} ,json 딕셔너리 갯수 {len(data4):4d}")

In [ ]:

import os

def find_images(root_dir):
    # 이미지 확장자 목록
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp'}
    
    image_files = []
    
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if os.path.splitext(file)[1].lower() in image_extensions:
                image_files.append(os.path.join(root, file))
    
    return image_files

In [ ]:
train_datas_paths = find_images(train_directory_path)
file_name = train_datas_paths[0].split('\\')[1]

In [ ]:

all_jasons = data1 + data2 + data3 + data4

In [ ]:

# 얼굴 감정 담기
label = []

for i in all_jasons:
    label.append(i['faceExp_uploader']) # 아래의 7가지 감정을 할 것인가 4가지 감정을 할 것인가 고민하다가 7가지로 정함. 
    # label.append(i['annot_A']['faceExp'])
    # label.append(i['annot_B']['faceExp'])
    # label.append(i['annot_C']['faceExp'])

In [ ]:
# 라벨에 모인 값을 중복값을 제외하고 보여줌
set(label)

In [ ]:
# 배경 값 담기
bgs = []

for i in all_jasons:
    bgs.append(i['annot_A']['bg'])
    bgs.append(i['annot_B']['bg'])
    bgs.append(i['annot_C']['bg'])

In [ ]:
# bgs에 모인 값을 중복값을 제외하고 보여줌
set(bgs)

In [ ]:
# 랜덤으로 사진 한 장 뽑기 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

i = random.randint(1, len(train_datas_paths)-2) 

name = all_jasons[i]['filename']
expr = all_jasons[i]['faceExp_uploader']
boxe = all_jasons[i]['annot_A']['boxes']
back = all_jasons[i]['annot_A']['bg']

min_x, min_y, max_x, max_y = [int(value) for value in boxe.values()] 
name, expr, min_x, min_y, max_x, max_y, back

for i in train_datas_paths:
    if name in i:
        img_path = i

image = cv2.imread(img_path)

cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (0, 255, 0), 10)
plt.axis('off')

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

plt.text(max_x, max_y, expr, color='white', fontsize=25,
         ha='left', va='top')


plt.show()

In [5]:
from ultralytics import YOLOv10

# model = YOLOv10('yolov10m')
# If you want to finetune the model with pretrained weights, you could load the 
# pretrained weights like below
# model = YOLOv10.from_pretrained('jameslahm/yolov10{n/s/m/b/l/x}')
# or
# wget https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10{n/s/m/b/l/x}.pt
# model = YOLOv10('yolov10{n/s/m/b/l/x}.pt')

# 훈련 코드
# model.train(data='coco.yaml', epochs=500, batch=256, imgsz=640)

# 예측 코드
model = YOLOv10.from_pretrained('jameslahm/yolov10s')
# yolo 10 프리트레인드 모델중에 좀더 많은 분류를 할 수 있는 모델 찾기

result=model.predict(image)

OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다. Error loading "c:\users\administrator\miniconda3\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
objects_form_pic = []
#result[0].boxes.cls는 탐지된 객체들이 가지고 있는 클래스의 인덱스
for i in result[0].boxes.cls:
    objects_form_pic.append(result[0].names[int(i)])

In [ ]:
objects_form_pic

In [ ]:
# result[0].boxes.conf는 해당 객체가 실제로 탐지된 클래스일 확률을 나타내는 신뢰도 점수
result[0].boxes.conf

In [ ]:
plt.imshow(result[0].plot())
plt.axis('off')  # 축 숨기기
plt.show()
back